In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = sc.read_h5ad('/Users/evanli/Documents/Research_datasets/PBMC_Hao/Hao_PBMC.h5ad')
data.shape

(161764, 20568)

In [3]:
# reverse log1p transformation
# np.expm1: exp(x) - 1 (is the inverse function of log1p)
data.X = np.expm1(data.X)

In [4]:
type(data.X)

scipy.sparse.csr.csr_matrix

In [5]:
data.obs.head(10)  # 161764

,nCount_ADT,nFeature_ADT,nCount_RNA,nFeature_RNA,orig.ident,lane,donor_id,time,celltype.l1,celltype.l2,...,tissue_ontology_term_id,suspension_type,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage
L1_AAACCCAAGAAACTCA,7430.0,221,10823.0,2915,P2_7,L1,P2,7,Mono,CD14 Mono,...,UBERON:0000178,cell,CD14-positive monocyte,10x 3' v3,normal,Homo sapiens,unknown,blood,unknown,unknown
L1_AAACCCAAGACATACA,5949.0,211,5864.0,1617,P1_7,L1,P1,7,CD4 T,CD4 TCM,...,UBERON:0000178,cell,"central memory CD4-positive, alpha-beta T cell",10x 3' v3,normal,Homo sapiens,unknown,blood,unknown,unknown
L1_AAACCCACAACTGGTT,6547.0,217,5067.0,1381,P4_3,L1,P4,3,CD8 T,CD8 Naive,...,UBERON:0000178,cell,"naive thymus-derived CD8-positive, alpha-beta ...",10x 3' v3,normal,Homo sapiens,unknown,blood,unknown,unknown
L1_AAACCCACACGTACTA,3508.0,207,4786.0,1890,P3_7,L1,P3,7,NK,NK,...,UBERON:0000178,cell,natural killer cell,10x 3' v3,normal,Homo sapiens,unknown,blood,unknown,unknown
L1_AAACCCACAGCATACT,6318.0,219,6505.0,1621,P4_7,L1,P4,7,CD8 T,CD8 Naive,...,UBERON:0000178,cell,"naive thymus-derived CD8-positive, alpha-beta ...",10x 3' v3,normal,Homo sapiens,unknown,blood,unknown,unknown
L1_AAACCCACATCAGTCA,5195.0,213,4332.0,1633,P3_3,L1,P3,3,CD8 T,CD8 TEM,...,UBERON:0000178,cell,"effector memory CD8-positive, alpha-beta T cell",10x 3' v3,normal,Homo sapiens,unknown,blood,unknown,unknown
L1_AAACCCACATCGGTTA,7507.0,223,9333.0,2672,P1_7,L1,P1,7,Mono,CD16 Mono,...,UBERON:0000178,cell,"CD14-low, CD16-positive monocyte",10x 3' v3,normal,Homo sapiens,unknown,blood,unknown,unknown
L1_AAACCCACATGGATCT,8138.0,226,3589.0,1122,P4_3,L1,P4,3,B,B intermediate,...,UBERON:0000178,cell,B cell,10x 3' v3,normal,Homo sapiens,unknown,blood,unknown,unknown
L1_AAACCCAGTGGAACAC,2858.0,205,3251.0,1375,P1_3,L1,P1,3,NK,NK,...,UBERON:0000178,cell,natural killer cell,10x 3' v3,normal,Homo sapiens,unknown,blood,unknown,unknown
L1_AAACCCATCCACACCT,4501.0,210,3401.0,1200,P3_3,L1,P3,3,CD8 T,CD8 Naive,...,UBERON:0000178,cell,"naive thymus-derived CD8-positive, alpha-beta ...",10x 3' v3,normal,Homo sapiens,unknown,blood,unknown,unknown


In [6]:
# row sum of data.X
row_sum = data.X.sum(axis=1)
row_sum[:10]  # equal to total_counts (not integer)

matrix([[6379.],
        [5692.],
        [5059.],
        [4979.],
        [5854.],
        [4934.],
        [6378.],
        [4489.],
        [4366.],
        [4722.]], dtype=float32)

In [7]:
row_sum.shape

(161764, 1)

In [8]:
# assuming data.X is your sparse CSR matrix
num_nonzero = data.X.getnnz(axis=1)
num_nonzero[:10]

array([2547, 1614, 1375, 1884, 1620, 1628, 2543, 1123, 1376, 1215],
      dtype=int32)

In [9]:
# QC metrics
sc.pp.calculate_qc_metrics(data, inplace=True)

In [10]:
data.obs.head(10)

,nCount_ADT,nFeature_ADT,nCount_RNA,nFeature_RNA,orig.ident,lane,donor_id,time,celltype.l1,celltype.l2,...,self_reported_ethnicity,development_stage,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes
L1_AAACCCAAGAAACTCA,7430.0,221,10823.0,2915,P2_7,L1,P2,7,Mono,CD14 Mono,...,unknown,unknown,2547,7.843064,6379.0,8.760923,33.531902,44.348644,54.350212,66.138893
L1_AAACCCAAGACATACA,5949.0,211,5864.0,1617,P1_7,L1,P1,7,CD4 T,CD4 TCM,...,unknown,unknown,1614,7.387090,5692.0,8.646993,44.448349,59.715390,68.499649,80.288124
L1_AAACCCACAACTGGTT,6547.0,217,5067.0,1381,P4_3,L1,P4,3,CD8 T,CD8 Naive,...,unknown,unknown,1375,7.226936,5059.0,8.529121,46.590235,62.976873,71.496343,82.704092
L1_AAACCCACACGTACTA,3508.0,207,4786.0,1890,P3_7,L1,P3,7,NK,NK,...,unknown,unknown,1884,7.541683,4979.0,8.513186,32.817835,44.466760,54.729865,69.250854
L1_AAACCCACAGCATACT,6318.0,219,6505.0,1621,P4_7,L1,P4,7,CD8 T,CD8 Naive,...,unknown,unknown,1620,7.390799,5854.0,8.675052,44.055347,61.120601,69.149300,80.645712
L1_AAACCCACATCAGTCA,5195.0,213,4332.0,1633,P3_3,L1,P3,3,CD8 T,CD8 TEM,...,unknown,unknown,1628,7.395722,4934.0,8.504107,37.474666,51.155249,62.910418,77.117957
L1_AAACCCACATCGGTTA,7507.0,223,9333.0,2672,P1_7,L1,P1,7,Mono,CD16 Mono,...,unknown,unknown,2543,7.841493,6378.0,8.760767,32.188774,43.571653,54.374412,66.556914
L1_AAACCCACATGGATCT,8138.0,226,3589.0,1122,P4_3,L1,P4,3,B,B intermediate,...,unknown,unknown,1123,7.024649,4489.0,8.409608,45.622633,62.396970,73.401648,86.121631
L1_AAACCCAGTGGAACAC,2858.0,205,3251.0,1375,P1_3,L1,P1,3,NK,NK,...,unknown,unknown,1376,7.227662,4366.0,8.381831,35.524508,46.930829,59.138800,78.401283
L1_AAACCCATCCACACCT,4501.0,210,3401.0,1200,P3_3,L1,P3,3,CD8 T,CD8 Naive,...,unknown,unknown,1215,7.103322,4722.0,8.460199,43.752647,58.788649,69.737399,84.858111


## Garnett scaling on adata.X (not log transformed)

In [14]:
from scipy.stats import gmean
# a: total read counts per cell
a = data.obs['total_counts']

# assuming data.X is your sparse CSR matrix
D = gmean(a) / np.median(data.X.getnnz(axis=1))

In [15]:
D

3.3108369956438457

In [16]:
# Transform expression matrix M to N
M = data.X
median_g = np.median(M.getnnz(axis=1))
# total read counts per cell / D * median_g
f_j = M.sum(axis=1).A1 / (D * median_g)

# transformed N
N = M / f_j[:, None]  # convert 1D array f_j to 2D array

In [17]:
f_j.shape

(161764,)

In [19]:
f_j[:10]

array([0.95239896, 0.84982836, 0.75532   , 0.74337584, 0.8740153 ,
       0.73665726, 0.95224965, 0.67021775, 0.65185356, 0.70500517],
      dtype=float32)

In [27]:
f_j[-5:]

array([1.2278616, 1.2938532, 1.2760862, 1.3383453, 1.2750411],
      dtype=float32)

In [20]:
type(M)

scipy.sparse.csr.csr_matrix

In [21]:
type(N)

numpy.matrix

In [22]:
M.shape

(161764, 20568)

In [23]:
N.shape

(161764, 20568)

In [26]:
M.toarray()

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [25]:
N

matrix([[0.       , 1.0499802, 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        ...,
        [0.       , 0.7836461, 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.7471913, ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ]], dtype=float32)

In [28]:
# turn into function
def garnett_scaling(M):
    # M is the original expression matrix, in sparse CSR format
    # transform expression matrix M to N

    # a: total read counts per cell
    a = data.obs['total_counts']

    # calculate D from M, or take D as input??
    D_size_factor = gmean(a) / np.median(M.getnnz(axis=1))

    median_g = np.median(M.getnnz(axis=1))
    f_j = M.sum(axis=1).A1 / (D_size_factor * median_g)
    N = M / f_j[:, None]
    
    return N